In [4]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os
import json
import cv2
import random
import sklearn
import pydot
import json
import tqdm
import shapely
from shapely.geometry import Polygon
from ultralytics import YOLO
from matplotlib import image as mpimg

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications, backend, layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.models import load_model as load_model_v2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Activation, Dropout, Flatten, Dense, GlobalMaxPool2D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam, Adamax, RMSprop
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score, classification_report

In [5]:
# Directory Initialization

#Training
training_image = "F:/UNI/Proposal Skripsi/deepfashion2_dataset/train/image"
training_annos = "F:/UNI/Proposal Skripsi/deepfashion2_dataset/train/annos"

#Validation
val_image = "F:/UNI/Proposal Skripsi/deepfashion2_dataset/validation/image"
val_annos = "F:/UNI/Proposal Skripsi/deepfashion2_dataset/validation/annos"

#Test
test_image = "F:/UNI/Proposal Skripsi/deepfashion2_dataset/test/image"

In [7]:
#Test if disk F: can be read
#If not please disconnect then reconnect Disk F:

img_id = "006349"
image = mpimg.imread(f"F:/UNI/Proposal Skripsi/deepfashion2_dataset/train/image/{img_id}.jpg")
plt.imshow(image)
plt.show()

AttributeError: module 'matplotlib' has no attribute 'imshow'

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = backend.sum(backend.round(backend.clip(y_true, 0, 1)))
    recall = true_positive / (possible_positives + backend.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = backend.sum(backend.round(backend.clip(y_pred, 0, 1)))
    precision = true_positive / (predicted_positives + backend.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    f1 = 2*((precision * recall)/(precision + recall + backend.epsilon()))
    return f1_score

def TrainValidInitiator(self):
    
    TrainGenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=360,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range=(0.2, 1.2),
        channel_shift_range=50,
        fill_mode='wrap',
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        rescale=1./255
    )
    
    ValidGenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    )
    
    return TrainGenerator, ValidGenerator

def DataFlowFromDirectory(self):
    
    TrainGenerator, ValidGenerator = self.TrainValidInitiator()
    
    train_data = TrainGenerator.flow_from_directory(
        training_image,
        targetsize = (64,64),
        color_mode='rgb',
        batch_size=self.BatchSize,
        shuffle=True,
        seed=12284,
    )
    
    val_data = TrainGenerator.flow_from_directory(
        val_image,
        targetsize = (64,64),
        color_mode='rgb',
        batch_size=self.BatchSize,
    )

def Optimizer(self, optimizers):
    if optimizers == "SGD":
        opt = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    elif optimizers == "RMS":
        opt = RMS(lr = 0.001, rho=0.9, epsilon=1e-8, decay = 0.0)
    elif optimizers == "ADAM":
        opt = Adam(lr=3e-4)
    return opt


def create_model():
    model = tf.keras.Sequential()
    # Must define the input shape in the first layer of the neural network
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', input_shape=(64,64,1))) 
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    #Compiling the model
    model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adamax',
             metrics=['accuracy'])
    
    return model
# Create a basic model instance
model = create_model()
model.summary()